In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as sco
from pylab import plt
plt.style.use("ggplot")

In [21]:
def find_covariance_matrix(corr, st_dev):
    n = len(st_dev)
    cov_matrix = np.zeros((n, n))
    
    # Indexing the correlations based on the provided order [ρ12, ρ23, ρ13]
    corr_index = { (0, 1): 0, (1, 2): 1, (0, 2): 2 }
    
    # Fill the upper triangle and the lower triangle symmetrically
    for i in range(n):
        for j in range(i + 1, n):
            rho = corr[corr_index[(i, j)] if (i, j) in corr_index else corr_index[(j, i)]]
            cov_matrix[i, j] = rho * st_dev[i] * st_dev[j]
            cov_matrix[j, i] = cov_matrix[i, j]

    # Fill the diagonal
    for i in range(n):
        cov_matrix[i, i] = st_dev[i] ** 2

    return cov_matrix

def invert_matrix(matrix):
    return np.linalg.inv(matrix)

def find_mvp_weights(inverted_cov_mat, st_dev):
    u = np.ones(len(st_dev))
    u_C_inv = u @ inverted_cov_mat
    u_C_inv_uT = u_C_inv @ u.T
    
    mvp_weights = []
    for i in u_C_inv:
        mvp_weights.append(i / u_C_inv_uT)
    
    return mvp_weights

In [22]:
risk_free_rate = 0.03

stocks = ["HSBA.L", "RR.L", "BLND.L"]
num_assets = len(stocks)

yf_stock_data = yf.download(" ".join(stocks), start="2023-01-01", end="2024-01-01", rounding=True)["Close"]
yf_stock_data.to_csv("test_stock_data.csv", index=True, header=True)

ftse_index = yf.download("^FTSE", start="2023-01-01", end="2024-01-01", progress=False, rounding=True)
ftse_index.to_csv("test_ftse_data.csv", index=True, header=True)

[                       0%%                      ]

[*********************100%%**********************]  3 of 3 completed


In [23]:
raw_stock_data = pd.DataFrame(yf_stock_data)
raw_stock_data.fillna(method="ffill", inplace=True)
clean_stock_data = raw_stock_data

raw_ftse_data = pd.DataFrame(ftse_index)
raw_ftse_data.fillna(method="ffill", inplace=True)
clean_ftse_data = raw_ftse_data

clean_stock_data.head(), clean_ftse_data.head()

/var/folders/qg/1zr492bn2zqcvxkm30bd16tc0000gn/T/ipykernel_10239/2942537575.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  raw_stock_data.fillna(method="ffill", inplace=True)
/var/folders/qg/1zr492bn2zqcvxkm30bd16tc0000gn/T/ipykernel_10239/2942537575.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  raw_ftse_data.fillna(method="ffill", inplace=True)


(Ticker      BLND.L  HSBA.L    RR.L
 Date                              
 2023-01-03   408.2   529.9   98.91
 2023-01-04   412.8   543.5  101.40
 2023-01-05   409.5   565.3  102.66
 2023-01-06   410.2   568.6  102.90
 2023-01-09   410.0   563.2  103.76,
               Open    High     Low   Close  Adj Close     Volume
 Date                                                            
 2023-01-03  7451.7  7626.4  7448.7  7554.1     7554.1  914070400
 2023-01-04  7554.1  7609.3  7545.2  7585.2     7585.2  753877000
 2023-01-05  7585.2  7658.8  7564.7  7633.5     7633.5  883850800
 2023-01-06  7633.5  7700.8  7633.5  7699.5     7699.5  813538800
 2023-01-09  7699.5  7725.7  7688.7  7724.9     7724.9  847858600)

In [24]:
stock_return = clean_stock_data.pct_change(periods=1)
stock_return.dropna(inplace=True)
stock_return.head()

Ticker,BLND.L,HSBA.L,RR.L
Date,,,
2023-01-04,0.011269,0.025665,0.025174
2023-01-05,-0.007994,0.040110,0.012426
2023-01-06,0.001709,0.005838,0.002338
2023-01-09,-0.000488,-0.009497,0.008358
2023-01-10,0.000488,0.001598,-0.026407


In [25]:
expected_returns = []
standard_deviations = []

for column in stock_return.columns:
    expected_returns.append(stock_return[column].mean())
    standard_deviations.append(stock_return[column].std())

print(expected_returns)
print(standard_deviations)

[0.00012285136296880773, 0.0008130606875553639, 0.0048300323581924745]
[0.020534158265751588, 0.013107892932961787, 0.028751285018817605]


In [26]:
corr_mat = clean_stock_data.corr()

corr_12 = corr_mat.iloc[0, 1]
corr_23 = corr_mat.iloc[1, 2]
corr_13 = corr_mat.iloc[0, 2]

correlations = [corr_12, corr_23, corr_13]


In [27]:
cov_mat = find_covariance_matrix(correlations, standard_deviations)
cov_mat_inv = invert_matrix(cov_mat)

print(cov_mat)
print("\n")
print(cov_mat_inv)

[[ 4.21651656e-04 -6.98949540e-05 -2.61215721e-04]
 [-6.98949540e-05  1.71816857e-04  1.28036680e-04]
 [-2.61215721e-04  1.28036680e-04  8.26636390e-04]]


[[2999.33690594  580.89265513  857.81172111]
 [ 580.89265513 6692.08231079 -852.96717462]
 [ 857.81172111 -852.96717462 1612.90382046]]


In [28]:
mvp_weights = find_mvp_weights(cov_mat_inv, standard_deviations)
print(mvp_weights)

[0.35573207270906915, 0.5145969884453986, 0.12967093884553224]


In [29]:
expected_returns_array = np.array(expected_returns)

mvp_return = mvp_weights @ expected_returns_array.T
print(mvp_return)

0.0010884155817644818
